In [1]:
def Glassdoor_scraping(url):
    
    import requests
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    
    list_ = [] ; f_r = []
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    results_page = BeautifulSoup(response.content,'html.parser')

    tag2 = results_page.find_all('h2', class_="h2 summary strong mt-0 mb-xsm")
    tag3 = results_page.find_all('p', class_="mainText mb-0")
    tag4 = results_page.find_all('p', class_="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge")
    ####get sub_attributes ratings 
    tag5 = results_page.find_all('div',class_='subRatings module stars__StarsStyles__subRatings')

    sub_ratings=[]
    for item in tag5: 
        all_tag = list(map(lambda x:x.get_text(), item.find('ul',class_='undecorated')))
        sub_rating= list(map(lambda x:x.get('title'),item.find_all('span',class_='gdBars gdRatings med')))
        
        if 'Work/Life Balance' not in all_tag:
            sub_rating.insert(0,np.nan)
            
        if 'Culture & Values' not in all_tag:
            sub_rating.insert(1,np.nan)

        if 'Career Opportunities' not in all_tag:
            sub_rating.insert(2,np.nan)

        if 'Compensation and Benefits' not in all_tag:
            sub_rating.insert(3,np.nan)
            
        if 'Senior Management' not in all_tag:
            sub_rating.insert(4,np.nan)
 
        sub_ratings.append(sub_rating)

    ###get subrating
    sub_rate=[]; i = 0
    tag6 = results_page.find_all('span',class_='gdStars gdRatings sm stars__StarsStyles__gdStars')
    for item in tag6:
        if item.find('i',class_='subRatingsDrop'):
            sub_rate.append(sub_ratings[i])
            i = i+1
        else:
            sub_rate.append([np.nan,np.nan,np.nan,np.nan,np.nan])
  

    title = list(map(lambda x:x.get_text(),tag2))
    working_time = list(map(lambda x:x.get_text(),tag3))
    Pros = list(map(lambda x:x.get_text(),[i for i in tag4 if not tag4.index(i)%2]))
    Cons = list(map(lambda x:x.get_text(),[i for i in tag4 if tag4.index(i)%2]))

    list_.append(pd.DataFrame(list(zip(title,working_time,Pros,Cons)),columns =['Title','WorkTime','Pros','Cons']))
    s_r = pd.DataFrame(sub_rate,columns=['Work/Life Balance','Culture & Values','Career Opportunities','Compensation & Benefits','Senior Management'])
    

    while(True):
        next_page_link = results_page.find('li', class_="pagination__PaginationStyle__next").find('a').get('href')
        print(next_page_link)

####改动这里的页数！！如果想要爬几百页就break一次的话！！！！        
#         if next_page_link == '/Reviews/Microsoft-Reviews-E1651_P94.htm':
#             break
        
        response = requests.get('https://www.glassdoor.com'+next_page_link, headers=headers)
        results_page = BeautifulSoup(response.content,'html.parser')

        tag2 = results_page.find_all('h2', class_="h2 summary strong mt-0 mb-xsm")
        tag3 = results_page.find_all('p', class_="mainText mb-0")
        tag4 = results_page.find_all('p', class_="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge")
        
        ####get sub_attributes ratings 
        tag5 = results_page.find_all('div',class_='subRatings module stars__StarsStyles__subRatings')

        sub_ratings=[]
        for item in tag5: 
            all_tag = list(map(lambda x:x.get_text(), item.find('ul',class_='undecorated')))
            sub_rating= list(map(lambda x:x.get('title'),item.find_all('span',class_='gdBars gdRatings med')))

            if 'Work/Life Balance' not in all_tag:
                sub_rating.insert(0,np.nan)

            if 'Culture & Values' not in all_tag:
                sub_rating.insert(1,np.nan)

            if 'Career Opportunities' not in all_tag:
                sub_rating.insert(2,np.nan)

            if 'Compensation and Benefits' not in all_tag:
                sub_rating.insert(3,np.nan)

            if 'Senior Management' not in all_tag:
                sub_rating.insert(4,np.nan)

            sub_ratings.append(sub_rating)

        ###get subrating
        sub_rate=[]; i = 0
        tag6 = results_page.find_all('span',class_='gdStars gdRatings sm stars__StarsStyles__gdStars')
        for item in tag6:
            if item.find('i',class_='subRatingsDrop'):
                sub_rate.append(sub_ratings[i])
                i = i+1
            else:
                sub_rate.append([np.nan,np.nan,np.nan,np.nan,np.nan])
        
        
        title = list(map(lambda x:x.get_text(),tag2))
        working_time = list(map(lambda x:x.get_text(),tag3))
        Pros = list(map(lambda x:x.get_text(),[i for i in tag4 if not tag4.index(i)%2]))
        Cons = list(map(lambda x:x.get_text(),[i for i in tag4 if tag4.index(i)%2]))

       
        if title:
            list_.append(pd.DataFrame(list(zip(title,working_time,Pros,Cons)),columns =['Title','WorkTime','Pros','Cons']))
            s_r = pd.concat([s_r, pd.DataFrame(sub_rate,columns=['Work/Life Balance','Culture & Values','Career Opportunities','Compensation & Benefits','Senior Management'])])
        else:
            break
        
    
    result = pd.concat(list_)
    result = result.reset_index(drop = True)
    s_r = s_r.reset_index(drop=True)

    final_result = pd.concat([result,s_r],axis=1)

    return final_result

In [2]:
apple = Glassdoor_scraping(url = 'https://www.glassdoor.com/Reviews/Apple-Reviews-E1138.htm')

/Reviews/Apple-Reviews-E1138_P2.htm
/Reviews/Apple-Reviews-E1138_P3.htm
/Reviews/Apple-Reviews-E1138_P4.htm
/Reviews/Apple-Reviews-E1138_P5.htm
/Reviews/Apple-Reviews-E1138_P6.htm
/Reviews/Apple-Reviews-E1138_P7.htm
/Reviews/Apple-Reviews-E1138_P8.htm
/Reviews/Apple-Reviews-E1138_P9.htm
/Reviews/Apple-Reviews-E1138_P10.htm
/Reviews/Apple-Reviews-E1138_P11.htm
/Reviews/Apple-Reviews-E1138_P12.htm
/Reviews/Apple-Reviews-E1138_P13.htm
/Reviews/Apple-Reviews-E1138_P14.htm
/Reviews/Apple-Reviews-E1138_P15.htm
/Reviews/Apple-Reviews-E1138_P16.htm
/Reviews/Apple-Reviews-E1138_P17.htm
/Reviews/Apple-Reviews-E1138_P18.htm
/Reviews/Apple-Reviews-E1138_P19.htm
/Reviews/Apple-Reviews-E1138_P20.htm
/Reviews/Apple-Reviews-E1138_P21.htm
/Reviews/Apple-Reviews-E1138_P22.htm
/Reviews/Apple-Reviews-E1138_P23.htm
/Reviews/Apple-Reviews-E1138_P24.htm
/Reviews/Apple-Reviews-E1138_P25.htm
/Reviews/Apple-Reviews-E1138_P26.htm
/Reviews/Apple-Reviews-E1138_P27.htm
/Reviews/Apple-Reviews-E1138_P28.htm
/Reviews/

/Reviews/Apple-Reviews-E1138_P221.htm
/Reviews/Apple-Reviews-E1138_P222.htm
/Reviews/Apple-Reviews-E1138_P223.htm
/Reviews/Apple-Reviews-E1138_P224.htm
/Reviews/Apple-Reviews-E1138_P225.htm
/Reviews/Apple-Reviews-E1138_P226.htm
/Reviews/Apple-Reviews-E1138_P227.htm
/Reviews/Apple-Reviews-E1138_P228.htm
/Reviews/Apple-Reviews-E1138_P229.htm
/Reviews/Apple-Reviews-E1138_P230.htm
/Reviews/Apple-Reviews-E1138_P231.htm
/Reviews/Apple-Reviews-E1138_P232.htm
/Reviews/Apple-Reviews-E1138_P233.htm
/Reviews/Apple-Reviews-E1138_P234.htm
/Reviews/Apple-Reviews-E1138_P235.htm
/Reviews/Apple-Reviews-E1138_P236.htm
/Reviews/Apple-Reviews-E1138_P237.htm
/Reviews/Apple-Reviews-E1138_P238.htm
/Reviews/Apple-Reviews-E1138_P239.htm
/Reviews/Apple-Reviews-E1138_P240.htm
/Reviews/Apple-Reviews-E1138_P241.htm
/Reviews/Apple-Reviews-E1138_P242.htm
/Reviews/Apple-Reviews-E1138_P243.htm
/Reviews/Apple-Reviews-E1138_P244.htm
/Reviews/Apple-Reviews-E1138_P245.htm
/Reviews/Apple-Reviews-E1138_P246.htm
/Reviews/App

/Reviews/Apple-Reviews-E1138_P437.htm
/Reviews/Apple-Reviews-E1138_P438.htm
/Reviews/Apple-Reviews-E1138_P439.htm
/Reviews/Apple-Reviews-E1138_P440.htm
/Reviews/Apple-Reviews-E1138_P441.htm
/Reviews/Apple-Reviews-E1138_P442.htm
/Reviews/Apple-Reviews-E1138_P443.htm
/Reviews/Apple-Reviews-E1138_P444.htm
/Reviews/Apple-Reviews-E1138_P445.htm
/Reviews/Apple-Reviews-E1138_P446.htm
/Reviews/Apple-Reviews-E1138_P447.htm
/Reviews/Apple-Reviews-E1138_P448.htm
/Reviews/Apple-Reviews-E1138_P449.htm
/Reviews/Apple-Reviews-E1138_P450.htm
/Reviews/Apple-Reviews-E1138_P451.htm
/Reviews/Apple-Reviews-E1138_P452.htm
/Reviews/Apple-Reviews-E1138_P453.htm
/Reviews/Apple-Reviews-E1138_P454.htm
/Reviews/Apple-Reviews-E1138_P455.htm
/Reviews/Apple-Reviews-E1138_P456.htm
/Reviews/Apple-Reviews-E1138_P457.htm
/Reviews/Apple-Reviews-E1138_P458.htm
/Reviews/Apple-Reviews-E1138_P459.htm
/Reviews/Apple-Reviews-E1138_P460.htm
/Reviews/Apple-Reviews-E1138_P461.htm
/Reviews/Apple-Reviews-E1138_P462.htm
/Reviews/App

/Reviews/Apple-Reviews-E1138_P653.htm
/Reviews/Apple-Reviews-E1138_P654.htm
/Reviews/Apple-Reviews-E1138_P655.htm
/Reviews/Apple-Reviews-E1138_P656.htm
/Reviews/Apple-Reviews-E1138_P657.htm
/Reviews/Apple-Reviews-E1138_P658.htm
/Reviews/Apple-Reviews-E1138_P659.htm
/Reviews/Apple-Reviews-E1138_P660.htm
/Reviews/Apple-Reviews-E1138_P661.htm
/Reviews/Apple-Reviews-E1138_P662.htm
/Reviews/Apple-Reviews-E1138_P663.htm
/Reviews/Apple-Reviews-E1138_P664.htm
/Reviews/Apple-Reviews-E1138_P665.htm
/Reviews/Apple-Reviews-E1138_P666.htm
/Reviews/Apple-Reviews-E1138_P667.htm
/Reviews/Apple-Reviews-E1138_P668.htm
/Reviews/Apple-Reviews-E1138_P669.htm
/Reviews/Apple-Reviews-E1138_P670.htm
/Reviews/Apple-Reviews-E1138_P671.htm
/Reviews/Apple-Reviews-E1138_P672.htm
/Reviews/Apple-Reviews-E1138_P673.htm
/Reviews/Apple-Reviews-E1138_P674.htm
/Reviews/Apple-Reviews-E1138_P675.htm
/Reviews/Apple-Reviews-E1138_P676.htm
/Reviews/Apple-Reviews-E1138_P677.htm
/Reviews/Apple-Reviews-E1138_P678.htm
/Reviews/App

/Reviews/Apple-Reviews-E1138_P869.htm
/Reviews/Apple-Reviews-E1138_P870.htm
/Reviews/Apple-Reviews-E1138_P871.htm
/Reviews/Apple-Reviews-E1138_P872.htm
/Reviews/Apple-Reviews-E1138_P873.htm
/Reviews/Apple-Reviews-E1138_P874.htm
/Reviews/Apple-Reviews-E1138_P875.htm
/Reviews/Apple-Reviews-E1138_P876.htm
/Reviews/Apple-Reviews-E1138_P877.htm
/Reviews/Apple-Reviews-E1138_P878.htm
/Reviews/Apple-Reviews-E1138_P879.htm
/Reviews/Apple-Reviews-E1138_P880.htm
/Reviews/Apple-Reviews-E1138_P881.htm
/Reviews/Apple-Reviews-E1138_P882.htm
/Reviews/Apple-Reviews-E1138_P883.htm
/Reviews/Apple-Reviews-E1138_P884.htm
/Reviews/Apple-Reviews-E1138_P885.htm
/Reviews/Apple-Reviews-E1138_P886.htm
/Reviews/Apple-Reviews-E1138_P887.htm
/Reviews/Apple-Reviews-E1138_P888.htm
/Reviews/Apple-Reviews-E1138_P889.htm
/Reviews/Apple-Reviews-E1138_P890.htm
/Reviews/Apple-Reviews-E1138_P891.htm
/Reviews/Apple-Reviews-E1138_P892.htm
/Reviews/Apple-Reviews-E1138_P893.htm
/Reviews/Apple-Reviews-E1138_P894.htm
/Reviews/App

/Reviews/Apple-Reviews-E1138_P1083.htm
/Reviews/Apple-Reviews-E1138_P1084.htm
/Reviews/Apple-Reviews-E1138_P1085.htm
/Reviews/Apple-Reviews-E1138_P1086.htm
/Reviews/Apple-Reviews-E1138_P1087.htm
/Reviews/Apple-Reviews-E1138_P1088.htm
/Reviews/Apple-Reviews-E1138_P1089.htm
/Reviews/Apple-Reviews-E1138_P1090.htm
/Reviews/Apple-Reviews-E1138_P1091.htm
/Reviews/Apple-Reviews-E1138_P1092.htm
/Reviews/Apple-Reviews-E1138_P1093.htm
/Reviews/Apple-Reviews-E1138_P1094.htm
/Reviews/Apple-Reviews-E1138_P1095.htm
/Reviews/Apple-Reviews-E1138_P1096.htm
/Reviews/Apple-Reviews-E1138_P1097.htm
/Reviews/Apple-Reviews-E1138_P1098.htm
/Reviews/Apple-Reviews-E1138_P1099.htm
/Reviews/Apple-Reviews-E1138_P1100.htm
/Reviews/Apple-Reviews-E1138_P1101.htm
/Reviews/Apple-Reviews-E1138_P1102.htm
/Reviews/Apple-Reviews-E1138_P1103.htm
/Reviews/Apple-Reviews-E1138_P1104.htm
/Reviews/Apple-Reviews-E1138_P1105.htm
/Reviews/Apple-Reviews-E1138_P1106.htm
/Reviews/Apple-Reviews-E1138_P1107.htm
/Reviews/Apple-Reviews-E1

/Reviews/Apple-Reviews-E1138_P1294.htm
/Reviews/Apple-Reviews-E1138_P1295.htm
/Reviews/Apple-Reviews-E1138_P1296.htm
/Reviews/Apple-Reviews-E1138_P1297.htm
/Reviews/Apple-Reviews-E1138_P1298.htm
/Reviews/Apple-Reviews-E1138_P1299.htm
/Reviews/Apple-Reviews-E1138_P1300.htm
/Reviews/Apple-Reviews-E1138_P1301.htm
/Reviews/Apple-Reviews-E1138_P1302.htm
/Reviews/Apple-Reviews-E1138_P1303.htm
/Reviews/Apple-Reviews-E1138_P1304.htm
/Reviews/Apple-Reviews-E1138_P1305.htm
/Reviews/Apple-Reviews-E1138_P1306.htm
/Reviews/Apple-Reviews-E1138_P1307.htm
/Reviews/Apple-Reviews-E1138_P1308.htm
/Reviews/Apple-Reviews-E1138_P1309.htm
/Reviews/Apple-Reviews-E1138_P1310.htm
/Reviews/Apple-Reviews-E1138_P1311.htm
/Reviews/Apple-Reviews-E1138_P1312.htm
/Reviews/Apple-Reviews-E1138_P1313.htm
/Reviews/Apple-Reviews-E1138_P1314.htm
/Reviews/Apple-Reviews-E1138_P1315.htm
/Reviews/Apple-Reviews-E1138_P1316.htm
/Reviews/Apple-Reviews-E1138_P1317.htm
/Reviews/Apple-Reviews-E1138_P1318.htm
/Reviews/Apple-Reviews-E1

/Reviews/Apple-Reviews-E1138_P1505.htm
/Reviews/Apple-Reviews-E1138_P1506.htm
/Reviews/Apple-Reviews-E1138_P1507.htm
/Reviews/Apple-Reviews-E1138_P1508.htm
/Reviews/Apple-Reviews-E1138_P1509.htm
/Reviews/Apple-Reviews-E1138_P1510.htm
/Reviews/Apple-Reviews-E1138_P1511.htm
/Reviews/Apple-Reviews-E1138_P1512.htm
/Reviews/Apple-Reviews-E1138_P1513.htm
/Reviews/Apple-Reviews-E1138_P1514.htm
/Reviews/Apple-Reviews-E1138_P1515.htm
/Reviews/Apple-Reviews-E1138_P1516.htm
/Reviews/Apple-Reviews-E1138_P1517.htm
/Reviews/Apple-Reviews-E1138_P1518.htm
/Reviews/Apple-Reviews-E1138_P1519.htm
/Reviews/Apple-Reviews-E1138_P1520.htm
/Reviews/Apple-Reviews-E1138_P1521.htm
/Reviews/Apple-Reviews-E1138_P1522.htm
/Reviews/Apple-Reviews-E1138_P1523.htm
/Reviews/Apple-Reviews-E1138_P1524.htm
/Reviews/Apple-Reviews-E1138_P1525.htm
/Reviews/Apple-Reviews-E1138_P1526.htm
/Reviews/Apple-Reviews-E1138_P1527.htm
/Reviews/Apple-Reviews-E1138_P1528.htm
/Reviews/Apple-Reviews-E1138_P1529.htm
/Reviews/Apple-Reviews-E1

In [3]:
apple

,Title,WorkTime,Pros,Cons,Work/Life Balance,Culture & Values,Career Opportunities,Compensation & Benefits,Senior Management
0,"""Great Company to work for""",I have been working at Apple full-time for mor...,"Extremely good benefits from healthcare, PTO, ...",Nothing that sticks out vs other jobs I've had,5.0,5.0,5.0,5.0,5.0
1,"""Some great benefits!""",I have been working at Apple part-time,Great benefits!\r\nPretty flexible hours!\r\nG...,Holiday scheduling\r\nNot able to be around pe...,NaN,NaN,NaN,NaN,NaN
2,"""Challenging, Rewarding, but ZERO work/life ...",I have been working at Apple full-time for mor...,"We work with geniuses - in every department, ...",ZERO ZERO ZERO work/life balance. Exec...,1.0,3.0,3.0,3.0,3.0
3,"""Excellent People""",I worked at Apple part-time for more than 3 years,"-Coworkers are smart, fun, and supportive\r\n-...",-It's retail: customers can be very difficult....,5.0,4.0,4.0,3.0,5.0
4,"""Good for retail but not for long term""",I worked at Apple part-time for more than a year,Benefits are okay if you're full-time. Part ti...,Management is pretty manipulative for the most...,1.0,5.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...
15349,"""Great products, great engineers, good company...",I have been working at Apple,"Apple makes high-quality products, both inside...","Management is extremely secretive, even intern...",4.0,NaN,3.5,3.0,4.0
15350,"""Great products, great engineers, good company...",I have been working at Apple,"Apple makes high-quality products, both inside...","Management is extremely secretive, even intern...",4.0,NaN,3.5,3.0,4.0
15351,"""More than a salary""",I have been working at Apple,"A company with an inspiration, incredibly dyna...",You know nothing or very little about what's c...,2.5,NaN,4.0,4.0,3.5
15352,"""BAD PAY - Good Atmosphere - No advancement""",I have been working at Apple,Exciting products and fun events,Horrible Pay Rates - 2-3 percent salary increa...,2.0,NaN,1.0,1.5,1.5


In [4]:
apple.to_csv('apple.csv')

In [25]:
dup = apple[apple.duplicated(keep=False)]

In [29]:
apple_drop = apple.drop_duplicates().reset_index(drop=True)

In [30]:
apple_drop

,Title,WorkTime,Pros,Cons,Work/Life Balance,Culture & Values,Career Opportunities,Compensation & Benefits,Senior Management
0,"""Great Company to work for""",I have been working at Apple full-time for mor...,"Extremely good benefits from healthcare, PTO, ...",Nothing that sticks out vs other jobs I've had,5.0,5.0,5.0,5.0,5.0
1,"""Some great benefits!""",I have been working at Apple part-time,Great benefits!\r\nPretty flexible hours!\r\nG...,Holiday scheduling\r\nNot able to be around pe...,NaN,NaN,NaN,NaN,NaN
2,"""Challenging, Rewarding, but ZERO work/life ...",I have been working at Apple full-time for mor...,"We work with geniuses - in every department, ...",ZERO ZERO ZERO work/life balance. Exec...,1.0,3.0,3.0,3.0,3.0
3,"""Excellent People""",I worked at Apple part-time for more than 3 years,"-Coworkers are smart, fun, and supportive\r\n-...",-It's retail: customers can be very difficult....,5.0,4.0,4.0,3.0,5.0
4,"""Good for retail but not for long term""",I worked at Apple part-time for more than a year,Benefits are okay if you're full-time. Part ti...,Management is pretty manipulative for the most...,1.0,5.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...
15257,"""I love Apple""",I have been working at Apple,"Apple, Mac OS, Products, Steve Job",Social life.. it's hard to leave apple,4.0,NaN,4.5,4.0,4.5
15258,"""Great products, great engineers, good company...",I have been working at Apple,"Apple makes high-quality products, both inside...","Management is extremely secretive, even intern...",4.0,NaN,3.5,3.0,4.0
15259,"""More than a salary""",I have been working at Apple,"A company with an inspiration, incredibly dyna...",You know nothing or very little about what's c...,2.5,NaN,4.0,4.0,3.5
15260,"""BAD PAY - Good Atmosphere - No advancement""",I have been working at Apple,Exciting products and fun events,Horrible Pay Rates - 2-3 percent salary increa...,2.0,NaN,1.0,1.5,1.5


In [31]:
apple_drop[apple_drop.duplicated(keep=False)]

,Title,WorkTime,Pros,Cons,Work/Life Balance,Culture & Values,Career Opportunities,Compensation & Benefits,Senior Management


In [32]:
apple_drop.to_csv('apple.csv')

## NLP

In [17]:
import pandas as pd
import numpy as np

In [18]:
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP(r'./stanford-corenlp-full-2018-10-05')

In [19]:
Lenovo = pd.read_csv('IBM.csv')

In [20]:
Lenovo.head()

,Unnamed: 0,Title,WorkTime,Pros,Cons,Work/Life Balance,Culture & Values,Career Opportunities,Compensation & Benefits,Senior Management
0,0,"""great company""",I worked at IBM full-time for more than 3 years,company treats it employees well,lots of employees and employees are not visable,4.0,4.0,4.0,4.0,4.0
1,1,"""Fair, strick ethic code and control, competit...",I have been working at IBM full-time for more ...,"Diversity, Women in Tech, AI, Product Design, ...","Competitive, hard to promote, too many layers",5.0,5.0,4.0,3.0,3.0
2,2,"""Good place to work""",I worked at IBM full-time for more than 8 years,"Pay, incentives and people, opportunities","Instability, career planning and bonus structures",2.0,1.0,5.0,3.0,3.0
3,3,"""Forward Thinking Company""",I have been working at IBM full-time for more ...,IBM's benefits are great and employees are val...,Switching to bi-weekly payroll would be a nice...,3.0,5.0,4.0,4.0,4.0
4,4,"""Great Company""",I have been working at IBM full-time for more ...,IBM is very innovative and a leader in many te...,May be over extended,4.0,5.0,5.0,4.0,5.0


In [21]:
pro = Lenovo['Pros']

In [22]:
con = Lenovo['Cons']

### Single Word Dictionary

In [23]:
def build_dic(data):
    keyWords_dp = dict()
    keyWords_pos = dict()
    for sent in data:
        After_parsing = nlp.dependency_parse(sent)
        print(After_parsing)
        tokenize = nlp.word_tokenize(sent)
        print(tokenize)
        pos_tag = nlp.pos_tag(sent)
        print(pos_tag)
        
        for i in range(len(After_parsing)):
            if keyWords_dp.get(After_parsing[i][0]):
                keyWords_dp[After_parsing[i][0]].append(tokenize[i])
            if not keyWords_dp.get(After_parsing[i][0]):
                keyWords_dp[After_parsing[i][0]] = [tokenize[i]]
            if keyWords_pos.get(pos_tag[i][1]):
                keyWords_pos[pos_tag[i][1]].append(tokenize[i])
            if not keyWords_pos.get(pos_tag[i][1]):
                keyWords_pos[pos_tag[i][1]]=[tokenize[i]]

    return keyWords_dp, keyWords_pos

In [24]:
dict1, dict2 = build_dic(pro[:10])

[('ROOT', 0, 2), ('nsubj', 2, 1), ('nsubj', 4, 3), ('ccomp', 2, 4), ('advmod', 4, 5)]
['company', 'treats', 'it', 'employees', 'well']
[('company', 'NN'), ('treats', 'VBZ'), ('it', 'PRP'), ('employees', 'NNS'), ('well', 'RB')]
[('ROOT', 0, 1), ('punct', 1, 2), ('root', 2, 3), ('case', 5, 4), ('nmod', 3, 5), ('punct', 5, 6), ('conj', 5, 7), ('punct', 5, 8), ('compound', 10, 9), ('conj', 5, 10), ('punct', 5, 11), ('compound', 13, 12), ('conj', 5, 13), ('punct', 5, 14), ('conj', 5, 15), ('case', 17, 16), ('nmod', 15, 17), ('punct', 5, 18), ('appos', 5, 19), ('case', 23, 20), ('det', 23, 21), ('amod', 23, 22), ('nmod', 19, 23), ('case', 26, 24), ('nmod:poss', 26, 25), ('nmod', 23, 26), ('punct', 3, 27)]
['Diversity', ',', 'Women', 'in', 'Tech', ',', 'AI', ',', 'Product', 'Design', ',', 'Offering', 'Management', ',', 'Voice', 'of', 'Customers', ',', 'care', 'about', 'the', 'overall', 'success', 'of', 'our', 'customers', '.']
[('Diversity', 'NN'), (',', ','), ('Women', 'NNS'), ('in', 'IN'), 

In [25]:
pro[:10]

0                     company treats it employees well
1    Diversity, Women in Tech, AI, Product Design, ...
2            Pay, incentives and people, opportunities
3    IBM's benefits are great and employees are val...
4    IBM is very innovative and a leader in many te...
5                 remote working from home was allowed
6            Low Salary as compared to other companies
7          Makes sure you have all the training needed
8                          Work life balance is a plus
9    It's a really supportive environment with unli...
Name: Pros, dtype: object

In [26]:
dict1

{'ROOT': ['company',
  'Diversity',
  'Pay',
  'IBM',
  'IBM',
  'remote',
  'Low',
  'Makes',
  'Work',
  'It'],
 'nsubj': ['treats', 'it', 'are', 'is', 'you', 'is', "'s"],
 'ccomp': ['employees', 'have'],
 'advmod': ['well', 'innovative', 'plus', 'supportive'],
 'punct': [',', ',', ',', ',', ',', ',', '.', ',', ',', '.', '.', '.'],
 'root': ['Women'],
 'case': ['in',
  'of',
  'about',
  'of',
  'benefits',
  'in',
  'home',
  'to',
  'with',
  'for'],
 'nmod': ['Tech',
  'Customers',
  'success',
  'customers',
  'corridors',
  'was',
  'companies',
  'opportunities',
  'training'],
 'conj': ['AI',
  'Design',
  'Management',
  'Voice',
  'incentives',
  'people',
  'opportunities',
  'valued',
  'supported',
  'leader',
  'betterment'],
 'compound': ['Product', 'Offering', 'technology', 'life', 'balance'],
 'appos': ['care'],
 'det': ['the', 'a', 'the', 'really'],
 'amod': ['overall',
  'many',
  'Salary',
  'other',
  'environment',
  'unlimited',
  'personal'],
 'nmod:poss': ['ou

In [27]:
dict2


{'NN': ['company',
  'Diversity',
  'care',
  'success',
  'leader',
  'technology',
  'home',
  'Salary',
  'training',
  'Work',
  'life',
  'balance',
  'environment',
  'training',
  'betterment'],
 'VBZ': ['treats', 'is', 'Makes', 'is', "'s"],
 'PRP': ['it', 'you', 'It'],
 'NNS': ['employees',
  'Women',
  'Customers',
  'customers',
  'incentives',
  'people',
  'opportunities',
  'benefits',
  'employees',
  'corridors',
  'companies',
  'opportunities'],
 'RB': ['well', 'very', 'really'],
 ',': [',', ',', ',', ',', ',', ',', ',', ','],
 'IN': ['in', 'of', 'about', 'of', 'in', 'from', 'as', 'with', 'for'],
 'NNP': ['Tech',
  'AI',
  'Product',
  'Design',
  'Offering',
  'Management',
  'Voice',
  'IBM',
  'IBM'],
 'DT': ['the', 'a', 'the', 'a', 'a'],
 'JJ': ['overall',
  'great',
  'innovative',
  'many',
  'remote',
  'Low',
  'other',
  'sure',
  'supportive',
  'unlimited',
  'personal'],
 'PRP$': ['our'],
 '.': ['.', '.', '.', '.'],
 'VB': ['Pay'],
 'CC': ['and', 'and', 'an

### Phrase Extraction

In [28]:
import re
from nltk.tokenize import sent_tokenize, word_tokenize

In [29]:
data = sent_tokenize(con[13])
data
list_=[]
for sent in data:
    print(sent)
    content = nlp.parse(sent)
    print(content)
    pattern = r'(\(NP [\(A-Za-z0-9 \)]+\))'
    match=re.findall(pattern,content)
    for item in match:
        item=re.sub(r'\(NP','',item)
        item=re.sub(r'\(','',item)
        item=re.sub(r'\)','',item)
        item=re.sub(r'[A-Z]{2,4}','',item)
        item=item.split()
#         print(item)
#         item =item[1::2]
        for i, j in enumerate(item):
            if len(j)==1:
                item.pop(i)
        item=' '.join(item)
        list_.append(item)
list_

Can be hard to navigate internal politics
(ROOT
  (SINV
    (VP (MD Can)
      (VP (VB be)
        (ADJP (JJ hard)
          (S
            (VP (TO to)
              (VP (VB navigate)))))))
    (NP (JJ internal) (NNS politics))))


['internal politics']

### Functionlize

In [30]:
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP(r'./stanford-corenlp-full-2018-10-05')

def get_phrase_list(sents):
    import re
    #print(sent)
    data = sent_tokenize(sents)
    list_=[]
    for sent in data:
        sent = sent.lower()
        sent = re.sub(r'\.',',',sent)#去除.->,
        content = nlp.parse(sent)
    #     print(content)
    #     pattern = r'(\(NP[ |\n][\(A-Za-z0-9 \)\n?]+\))'
        pattern = r'(\(NP [\(A-Za-z0-9 \)]+\))'
        match=re.findall(pattern,content)
        for item in match:
            item=re.sub(r'\(NP','',item)
            item=re.sub(r'\(','',item)
            item=re.sub(r'\)','',item)
            item=re.sub(r'[A-Z]{2,4}','',item)
            item=item.split()
    #         print(item)
    #         item =item[1::2]
            for i, j in enumerate(item):
                if len(j)==1:
                    item.pop(i)
            item=' '.join(item)
            list_.append(item)
    return list_

In [31]:
get_phrase_list(pro[0])

['company', 'it', 'employees']

In [32]:
for i in range(0,5):
    print(get_phrase_list(pro[i]))

['company', 'it', 'employees']
['diversity', 'women', 'tech', 'product design', 'management', 'voice', 'customers', 'care', 'the overall success']
['pay', 'opportunities']
['ibm', 'employees']
['ibm', 'leader', 'many technology corridors']


In [33]:
list__ =list()
for i in range(0,5):
    list__.extend(get_phrase_list(pro[i]))
list__

['company',
 'it',
 'employees',
 'diversity',
 'women',
 'tech',
 'product design',
 'management',
 'voice',
 'customers',
 'care',
 'the overall success',
 'pay',
 'opportunities',
 'ibm',
 'employees',
 'ibm',
 'leader',
 'many technology corridors']

In [34]:
import pandas as pd

def phrase_extraction(file_name):
    company = pd.read_csv(file_name)
    pro = company['Pros']
    #pro = pro[:20]
    con = company['Cons']
    #con = con[:20]
    pro_phrase_list = list(); pro_except=list()
    con_phrase_list = list(); con_except=list()
    for pro_sent in pro:
        try:
            pro_phrase_list.extend(get_phrase_list(pro_sent))
        except:
            pro_except.append(pro_sent)
    for con_sent in con:
        try:
            con_phrase_list.extend(get_phrase_list(con_sent))
        except:
            con_except.append(con_sent)
#     # remove duplicates
#     pro_phrase_list = list(dict.fromkeys(pro_phrase_list))
#     con_phrase_list = list(dict.fromkeys(con_phrase_list))
    
    return pro_phrase_list, con_phrase_list, pro_except, con_except

In [35]:
pro, con, pe, ce = phrase_extraction('IBM.csv')

In [36]:
len(con)

42659

In [56]:
pe

[]

In [62]:
ce

[]

In [37]:
import pandas as pd
df = pd.DataFrame([pro,con])
df=df.T
df.rename(columns={0:'pro',1:'con'},inplace=True)

In [38]:
df.to_csv('IBM_phrase.csv')

In [39]:
df

,pro,con
0,company,lots
1,it,employees
2,employees,many layers
3,diversity,instability
4,women,career planning
...,...,...
42654,None,whom
42655,None,acquired companies
42656,None,all
42657,None,the red tape


In [63]:
pro

['culture',
 'interesting work',
 'business',
 'chances',
 'many technologies',
 'open culture',
 'good wlb',
 'massive amount',
 'data',
 'lot',
 'still big name company people',
 'the best environment',
 'great working environment',
 'good mentors',
 'platform',
 'profitability',
 'the company',
 'innovation',
 'new things',
 '',
 'very',
 'environment',
 'lots',
 'opportunity',
 'great peers',
 'great company',
 'so bloody',
 'professional',
 'organization',
 'methodology',
 'alicloud',
 'there',
 'many good business',
 'good products',
 'alibaba',
 'it',
 'global',
 'the opportunity',
 'chinese company',
 'me',
 'work',
 'smart people',
 'good salary',
 'good package',
 'good benefits',
 'friendly environment',
 'talented',
 'good working hours',
 'good company',
 'good costumer service',
 'good coworkers',
 'exciting initiatives',
 'fast',
 'roles',
 'faster growth',
 'good learning tools',
 'colleagues',
 'opportunities',
 'new stuff',
 'benifits',
 'good',
 'good opportunity',
 

In [67]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

pro = ' '.join(pro)
wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=3000,height=3000).generate(pro)


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

ValueError: We need at least 1 word to plot a word cloud, got 0.

In [1]:
import pandas as pd
x = pd.read_csv('alibaba_phrase.csv')


In [2]:
pro = x['pro']

In [3]:
pro = list(pro)

In [4]:
for i, item in enumerate(pro):
    pro[i]=str(item)
pro_str = ' '.join(pro)

In [6]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline


wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=30000,height=30000).generate(pro_str)


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

KeyboardInterrupt: 

In [5]:
wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=3000,height=3000).generate(pro)


plt.imshow(wordcloud)
plt.axis('off')
plt.show()

NameError: name 'WordCloud' is not defined

In [18]:
from collections import Counter
from nltk.corpus import stopwords
words = [word.lower() for word in pro_str.split()
                  if word not in stopwords.words() and word.isalpha()]
word_count = Counter(words)
#取非空的前八个最多单词
most_11 = [x for x in word_count.most_common(100) if x[0] != ""]
#直接取前八个最多单词
print(word_count.most_common(100))
print(most_11)
for i in most_11:
    print(i[0])

[('good', 72), ('company', 45), ('great', 32), ('work', 26), ('environment', 26), ('high', 21), ('nice', 20), ('many', 18), ('big', 18), ('culture', 16), ('business', 16), ('people', 14), ('team', 13), ('group', 13), ('working', 12), ('salary', 12), ('data', 11), ('huge', 11), ('place', 11), ('new', 10), ('companies', 10), ('things', 9), ('growth', 9), ('development', 9), ('tech', 9), ('learning', 8), ('colleagues', 8), ('opportunities', 8), ('excellent', 8), ('system', 8), ('internet', 8), ('experience', 8), ('pay', 8), ('best', 7), ('platform', 7), ('chinese', 7), ('benefits', 7), ('employees', 7), ('large', 7), ('challenge', 7), ('office', 7), ('technology', 7), ('atmosphere', 7), ('different', 7), ('opportunity', 6), ('organization', 6), ('alibaba', 6), ('package', 6), ('hours', 6), ('promotion', 6), ('strong', 6), ('internal', 6), ('user', 6), ('flexible', 6), ('career', 6), ('friendly', 5), ('china', 5), ('job', 5), ('industry', 5), ('growing', 5), ('biggest', 5), ('software', 5)

In [19]:
list(filter(lambda x: 'environment' in x, pro))

['the best environment',
 'great working environment',
 'environment',
 'friendly environment',
 'good working environment',
 'good work environment',
 'passionate environment',
 'good environment',
 'comfortable environment',
 'work environment',
 'convenient work environment',
 'fierce competition environment',
 'great environment',
 'positive work environment',
 'nice work environment',
 'the work environment',
 'modern chinese corporation work environment',
 'entrepreneurial environment',
 'the working environment',
 'dynamic environment',
 'good technical environment',
 'good living environment',
 'nice working environment',
 'high growth environment',
 'the whole environment',
 'very nice environment']